In [89]:
import json
pfile="predicates.json";
apfile="animation_profile.json"
f = open(pfile)
content = f.read()
predicates=json.loads(content)

f1=open(apfile)
content1=f1.read()
animation_profile=json.loads(content1)

In [90]:
randomcolorList=[{'r': 0.73, 'g': 1.0, 'b': 1.0, 'a': 1.0}, {'r': 0.4, 'g': 0.55, 'b': 0.55, 'a': 1.0}, {'r': 0.86, 'g': 0.86, 'b': 0.86, 'a': 1.0}, {'r': 1.0, 'g': 0.85, 'b': 0.73, 'a': 1.0}, {'r': 1.0, 'g': 0.98, 'b': 0.8, 'a': 1.0}, {'r': 0.94, 'g': 1.0, 'b': 0.94, 'a': 1.0}, {'r': 1.0, 'g': 0.89, 'b': 0.88, 'a': 1.0}, {'r': 0.0, 'g': 0.53, 'b': 0.55, 'a': 1.0}, {'r': 1.0, 'g': 0.5, 'b': 0.14, 'a': 1.0}, {'r': 1.0, 'g': 0.76, 'b': 0.76, 'a': 1.0}, {'r': 0.18, 'g': 0.31, 'b': 0.31, 'a': 1.0}, {'r': 0.5, 'g': 1.0, 'b': 0.83, 'a': 1.0}, {'r': 0.8, 'g': 0.52, 'b': 0.25, 'a': 1.0}, {'r': 0.93, 'g': 0.93, 'b': 0.0, 'a': 1.0}, {'r': 0.0, 'g': 0.0, 'b': 0.5, 'a': 1.0}, {'r': 0.41, 'g': 0.55, 'b': 0.41, 'a': 1.0}, {'r': 1.0, 'g': 0.76, 'b': 0.76, 'a': 1.0}, {'r': 0.0, 'g': 0.0, 'b': 1.0, 'a': 1.0}, {'r': 0.0, 'g': 0.55, 'b': 0.0, 'a': 1.0}]
import random
def randomColor():
    return random.choice(randomcolorList)

In [91]:
import copy
objectList=copy.deepcopy(predicates["objects"])
stages=copy.deepcopy(predicates["stages"])
predicates_rules=animation_profile["predicates_rules"]

In [92]:
def initialObjects(objectList,animation_profile):
    unsolvedObjects={}
    for objectname in objectList:
        unsolvedObjects[objectname]={}
        objType=animation_profile["objects"]["default"]
        #update the value for each 
        for objproperty in animation_profile["shape"][objType]:
            value=animation_profile["shape"][objType][objproperty]
            if value is not False:
                if value =="randomcolor":
                    unsolvedObjects[objectname][objproperty]=randomColor()
                    continue
                unsolvedObjects[objectname][objproperty]=value
            else:
                unsolvedObjects[objectname][objproperty]=False
        unsolvedObjects[objectname]["name"]=objectname
    return unsolvedObjects

In [93]:
objectsDic=initialObjects(objectList,animation_profile)

In [94]:
def initalspace(size):
    array=[]
    for i in range(size):
        array.append(i)
    return array
space = initalspace(len(objectList))

def distribute(obj,space,spacebtw,width,remove):
    if not remove:
        if obj in space:
            objindex=space.index(obj)
            return objindex*(width+spacebtw)
        else:
            for num in range(len(space)):
                if num==space[num]:
                    space[num]=obj
                    return num*(width+spacebtw)
    else:
        if obj in space:
            objindex=space.index(obj)
            space[objindex]=objindex
            return True
        else:
            return False

In [95]:
def check_rule_complete(predicate,objectsDic,predicates_rules):
    predicatename=predicate["name"]
    objectnamelist=predicate["objectNames"]
    predicate_rule=predicates_rules[predicatename]
    for rulename in predicate_rule["rules"]:
        rule=predicate_rule[rulename]
        for key in rule:
            #0 is on the left side of equation
            if key != "value" and key !="0":
                property_check=rule[key]
                objectname=objectnamelist[int(key)]
                if objectsDic[objectname][property_check] is False:
                    return False
    return True

In [96]:
def applypredicates(predicate,objectsDic,predicates_rules):
    pname=predicate["name"]
    objects=predicate["objectNames"]
    left=objects[0]
    for rulename in predicates_rules[pname]["rules"]:
        propertyname=predicates_rules[pname][rulename]["0"]
        value=predicates_rules[pname][rulename]["value"]
        rule=predicates_rules[pname][rulename]
        if "function" in value:
            if value["function"]=="distributex":
                objectsDic[left][propertyname]=distribute(left,space,20,80,False)
        elif "equal" in value:
            right_value=value["equal"]
            if right_value in rule:
                right_proterpy=rule[right_value]
                right_object=objects[int(right_value)]
                objectsDic[left][propertyname]=objectsDic[right_object][right_proterpy]
            else:
                objectsDic[left][propertyname]=right_value
                
        elif "add" in value:
            rightvalue=0
            for additem in value["add"]:
                if additem in rule:
                    right_property=rule[additem]
                    right_object=objects[int(additem)]
                    addvalue=objectsDic[right_object][right_property]
                    rightvalue+=addvalue
                else:
                    rightvalue+=additem
            objectsDic[left][propertyname]=rightvalue

In [97]:
def solvepredicates(predicates,objectsDic,predicates_rules):
    while (len(predicates)>0):
        predicate=predicates.pop(0)
        if predicate["name"] not in predicates_rules:
            continue
        if check_rule_complete(predicate,objectsDic,predicates_rules):
            applypredicates(predicate,objectsDic,predicates_rules)
        else:
            if len(predicates)==0:
                return False
            predicates.append(predicate)
    return True

In [98]:
def solveAllStages(stages,objectsDic,predicates_rules):
    result={}
    result["visualStages"]=[]
    for stage in stages:
        stageDic={}
        objectDicCopy=copy.deepcopy(objectsDic)
        predicates=stage["items"]
        solvepredicates(predicates,objectDicCopy,predicates_rules)
        stageDic["visualSprites"]=objectDicCopy
        result["visualStages"].append(stageDic)
    return result

In [99]:
result=solveAllStages(stages,objectsDic,predicates_rules)

In [100]:
# resultJson = json.dumps(result)
# print(resultJson)

In [101]:
def transfer(oneStep,initialobjects):
    # list that stores object name
    finalFile = {}
    temp = []
    # panel size
    panelWidth = 500
    panelHeight = 550
    # dict that stores new position info
    dictX = {"minX":"",
            "maxX":"",
            "minY":"",
            "maxY":""}
    # new position info
    minX = 0.0
    maxX = 0.0
    minY = 0.0
    maxY = 0.0

   
    # generate new json file
    for i,object in enumerate(initialobjects):
        temp.append(oneStep[object])
        # get all information about position
        xNum = oneStep[object]["x"]
        yNum = oneStep[object]["y"]
        width = oneStep[object]["width"]
        height = oneStep[object]["height"]
        # transfer the position info into position needed in Unity
        minX = xNum/panelWidth
        maxX = (xNum + width)/panelWidth
        minY = yNum/panelHeight
        maxY = (yNum + height)/panelHeight
        dictX["minX"] = round(minX,2)
        dictX["maxX"] = round(maxX,2)
        dictX["minY"] = round(minY,2)
        dictX["maxY"] = round(maxY,2)
        # update the old dict
        temp[i].update(dictX)
        finalFile["visualSprites"] = temp
    return finalFile


final = {"visualStages":[]}
oneStep = {}
spriteList = []
lists = result["visualStages"]
for i,step in enumerate(lists):
    oneStep = lists[i]["visualSprites"]
    spriteList.append(transfer(oneStep,objectList))
    final["visualStages"] = spriteList
jsonfile = json.dumps(final)
print(jsonfile)

{"visualStages": [{"visualSprites": [{"prefab": "Block", "showName": true, "x": 0, "y": 0, "color": {"r": 0.0, "g": 0.53, "b": 0.55, "a": 1.0}, "width": 80, "height": 40, "name": "a", "minX": 0.0, "maxX": 0.16, "minY": 0.0, "maxY": 0.07}, {"prefab": "Block", "showName": true, "x": 100, "y": 0, "color": {"r": 1.0, "g": 0.5, "b": 0.14, "a": 1.0}, "width": 80, "height": 40, "name": "b", "minX": 0.2, "maxX": 0.36, "minY": 0.0, "maxY": 0.07}, {"prefab": "Block", "showName": true, "x": 300, "y": 0, "color": {"r": 1.0, "g": 0.85, "b": 0.73, "a": 1.0}, "width": 80, "height": 40, "name": "c", "minX": 0.6, "maxX": 0.76, "minY": 0.0, "maxY": 0.07}, {"prefab": "Block", "showName": true, "x": 250, "y": 480, "color": {"r": 0.94, "g": 1.0, "b": 0.94, "a": 1.0}, "width": 80, "height": 40, "name": "d", "minX": 0.5, "maxX": 0.66, "minY": 0.87, "maxY": 0.95}, {"prefab": "Block", "showName": true, "x": 200, "y": 0, "color": {"r": 0.4, "g": 0.55, "b": 0.55, "a": 1.0}, "width": 80, "height": 40, "name": "e"